In [1]:
# imports
# import the necessary packages
import pandas as pd
import numpy as np
import random
random.seed(1)
import csv
import cv2
import os,re, codecs
import tensorflow.keras.metrics
from tqdm import tqdm
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from gensim.models import KeyedVectors
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.utils import to_categorical
from imutils import paths
from tensorflow.python.keras.layers.merge import Concatenate, Average, concatenate
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix, f1_score, precision_score, recall_score
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from numpy import asarray,zeros
import seaborn as sns
from sklearn.model_selection import KFold
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('always') 

/sda/rina_1921cs13/anaconda3/envs/newnisbert/lib/python3.6/site-packages/ipykernel/pylab/config.py:70: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [2]:
#Lines to run the code on GPU
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

from tensorflow.python.keras import backend as K
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
K.set_session(tf.compat.v1.Session(config=config))

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:18:00.0, compute capability: 7.5



In [5]:
artphotos_dir = './testImages_artphoto/'
dir_contents = os.listdir(artphotos_dir)
print(len(dir_contents))

808


In [8]:
from collections import Counter
count_images = 0
count_labels = Counter()
for img_file in dir_contents:
    if '.jpg' in img_file:
        img_label = img_file.split('_')[0]
        count_labels[img_label] += 1 
        count_images += 1
    else:
        print(img_file)
print(count_images) # 806 images are there in total
print(count_labels) # Printing labels count

ReadMe.rtf
.DS_Store
806
Counter({'sad': 166, 'fear': 115, 'excitement': 105, 'awe': 102, 'amusement': 101, 'anger': 77, 'disgust': 70, 'contentment': 70})


In [9]:
#TODO: Categorize emotions
# (Original) True Emotions - anticipation, sadness, joy and trust
# (Original) False Emotions - Fear, Disgust, Surprise
# (Here) True Emotions - sad, contenment (236)
true_emotions = ['sad', 'contentment'] # Label 1
# (Here) False Emotions - fear, excitement, awe, amusement, anger, disgust (570)
false_emotions = ['fear', 'excitement', 'awe', 'amusement', 'anger', 'disgust'] # Label 0

In [10]:
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG19,VGG16
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.imagenet_utils import preprocess_input

image_embed = []
label =[]
for img_file in dir_contents:
    if '.jpg' in img_file:
        img_path = os.path.join(artphotos_dir, img_file)
        img_label = img_file.split('_')[0]
        img = load_img(img_path, target_size=(224, 224))	
        img = img_to_array(img)
        img = preprocess_input(img)
        image_embed.append(img)
        if img_label in true_emotions:
            label.append(1)
        elif img_label in false_emotions:
            label.append(0)
        else:
            print("you missed something")
    else:
        print('Not processed', img_file)

data = np.array(image_embed)
label_arr = np.array(label)
print(data.shape)
print(label_arr.shape)

Not processed ReadMe.rtf
Not processed .DS_Store
(806, 224, 224, 3)
(806,)


In [11]:
# Saving the data
np.save(os.path.join(artphotos_dir, 'data.npy'), data)
np.save(os.path.join(artphotos_dir, 'label.npy'), label_arr)